<a href="https://colab.research.google.com/github/sumitdua10/Seq-Networks/blob/master/Seq_to_Seq_Attention_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving book2.txt to book2.txt
User uploaded file "book2.txt" with length 20298 bytes


In [5]:
import pandas as pd
import numpy as np

df = pd.read_csv(fn, delimiter = '\t')
print(df.shape)
print(df[:5])


(797, 4)
           X  Date  Month  Year
0  18/9/1970    18      9  1970
1  15/7/1968    15      7  1968
2  23/5/1963    23      5  1963
3  27/7/2007    27      7  2007
4  16/6/1995    16      6  1995


In [0]:
#Code to get reproducible results
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, LSTM, GRU, Input, Embedding, Flatten, RepeatVector, Dropout
from keras.layers import Reshape, Add, Multiply, Concatenate, GlobalMaxPooling2D, GlobalMaxPooling1D, Dot
from keras.activations import softmax
import random as rn
import tensorflow as tf

np.random.seed(42)
rn.seed(12345)

# Force TensorFlow to use single thread.
# Multiple threads are a potential source of non-reproducible results.
# For further details, see: https://stackoverflow.com/questions/42022950/

session_conf = tf.ConfigProto(intra_op_parallelism_threads=1,
                              inter_op_parallelism_threads=1)

from keras import backend as K
tf.set_random_seed(1234)

sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

In [46]:
data = open(fn, 'r').read()
data= data.lower()
chars = list(set(data))
#print(data[1:50])

data_size, VOCAB_LEN = len(data), len(chars)
print('There are %d total characters and %d unique characters(vocab length)in your data.' % (data_size, VOCAB_LEN))

#2. Global variables
MAX_INPUT_SEQ_LEN = len(max(df['X']))
OUTPUT_SIZE = 4 # For year only (4-digit)
m = df.shape[0]

print("Max Length ", MAX_INPUT_SEQ_LEN)

#3. Create a dictionary & vocab size
char_to_ix = { ch:i for i,ch in enumerate(sorted(chars)) }
ix_to_char = { i:ch for i,ch in enumerate(sorted(chars)) }
print("Integer to character dict mapping:\n",ix_to_char)
print("\n Character to Integer dict mapping:")
print(char_to_ix)

There are 19501 total characters and 40 unique characters(vocab length)in your data.
Max Length  29
Integer to character dict mapping:
 {0: '\t', 1: '\n', 2: ' ', 3: ',', 4: '-', 5: '/', 6: '0', 7: '1', 8: '2', 9: '3', 10: '4', 11: '5', 12: '6', 13: '7', 14: '8', 15: '9', 16: ':', 17: 'a', 18: 'b', 19: 'c', 20: 'd', 21: 'e', 22: 'f', 23: 'g', 24: 'h', 25: 'i', 26: 'j', 27: 'l', 28: 'm', 29: 'n', 30: 'o', 31: 'p', 32: 'r', 33: 's', 34: 't', 35: 'u', 36: 'v', 37: 'w', 38: 'x', 39: 'y'}

 Character to Integer dict mapping:
{'\t': 0, '\n': 1, ' ': 2, ',': 3, '-': 4, '/': 5, '0': 6, '1': 7, '2': 8, '3': 9, '4': 10, '5': 11, '6': 12, '7': 13, '8': 14, '9': 15, ':': 16, 'a': 17, 'b': 18, 'c': 19, 'd': 20, 'e': 21, 'f': 22, 'g': 23, 'h': 24, 'i': 25, 'j': 26, 'l': 27, 'm': 28, 'n': 29, 'o': 30, 'p': 31, 'r': 32, 's': 33, 't': 34, 'u': 35, 'v': 36, 'w': 37, 'x': 38, 'y': 39}


In [0]:
#5. Create on hot encoding of size x = m X SEQ_L X VOCAB_LEN and y = m X OUTPUT_SIZE * VOCAB_LEN

x = np.zeros(shape = (m, MAX_INPUT_SEQ_LEN, VOCAB_LEN))
y = np.zeros(shape=(m,OUTPUT_SIZE, VOCAB_LEN))

#decoder input. This will be used as Teacher enforcing during training. 
# It will be y shifted by one character so that y[0] goes as input to 2nd decoder cell and so on. First input to decoder cell will be 0
# and output h,c from encoder LSTM will be used as initial states for first cell of decoder model.
de_x  = np.zeros(shape=(m,OUTPUT_SIZE, VOCAB_LEN))

df['X'] = df['X'].str.lower()
for i in range(m):
    for j,k in enumerate(df['X'][i]):
      try:
        x[i][j][char_to_ix[k]] = 1
      except:
        print("i {} j {} k {}".format( i, j, k))
        
    for a,b in enumerate(str(df['Year'][i])):
      try:
        
        y[i][a][char_to_ix[b]] = 1
        
        
        if a<OUTPUT_SIZE-1:
          de_x[i][a+1][char_to_ix[b]] = 1
        
      except:
        print("i {} a {} b {}".format( i, a, b))


In [55]:
#Test the encoding:

print(df['X'][0])
pos = np.argmax(x[0][0])
print("First character should match with first character of date")
print(ix_to_char[pos])

18/9/1970
First character should match with first character of date
1


In [186]:
activation_size=96
nn_units = 64
from keras.layers import Bidirectional
from keras import optimizers
import keras.backend as K

en_inputs = Input(shape=(MAX_INPUT_SEQ_LEN, VOCAB_LEN,))

fn = Bidirectional(LSTM(activation_size,  activation='tanh', return_state=True, return_sequences = True, dropout = 0.5))(en_inputs)
print(len(fn))
for mm in fn:
   print(mm.shape)
o = fn[0]
h = fn[1]
c = fn[2]
print(" o shape of encoder ", o.shape)
print(" h shape of encoder ", h.shape)

initial_o_input = Input(shape=(activation_size,))
initial_o = initial_o_input
initial_h = initial_o_input
initial_c = initial_o_input
#initial_o = initial_o_input
print("s-1 shape of decoder ", initial_s)

def softmax_over_time(x):
  assert(K.ndim(x) > 2)
  e = K.exp(x - K.max(x, axis=1, keepdims=True))
  s = K.sum(e, axis=1, keepdims=True)
  return e / s
  return softmax(x,axis=-2)

output= []


#repeat = RepeatVector(MAX_INPUT_SEQ_LEN)
#concats = Concatenate()
#dense_20_param = Dense(20, activation ='relu', name = "dense_20_param_{}".format(i))
#dense_1_param = Dense(1, activation=softmax_over_time, name="dense_1_param_{}".format(i))

def one_step_attention():
 
  initial_s = RepeatVector(MAX_INPUT_SEQ_LEN)(initial_h)
  #initial_s = repeat(initial_h)  
  concat = Concatenate()([initial_s, o])
  #concat = concats([initial_s,o])
  
  print("O shape before going into NN ", concat)
  
  nn = Dense(20, activation ='relu', name = "dense_20_param_{}".format(i))(concat)
  #nn = dense_20_param(concat)
  print("nn after 20 dense", nn )
  nn = Dropout(0.5)(nn)
  
  #nn = Dense(1, activation=softmax_over_time)(nn)
  nn = Dense(1, activation=softmax_over_time, name="dense_1_param_{}".format(i))(nn)
  #nn = dense_1_param(nn)
  #print("Neural network output ", nn)
  #nn = softmax(nn, axis=-2)
  print("After appplying nuerual network ", nn)
  print(" o shape before dot ", o.shape)
  context = Dot(axes=1)([nn,o])
  print(" context shape after dot ", context.shape)
  #context = Reshape((activation_size,))(context)
  #print(" context after reshape ", context.shape)
  return context

for i in range(OUTPUT_SIZE):

  context = one_step_attention()
  initial_oo = Reshape((1,activation_size,))(initial_o)
  decoder_o = Concatenate()([context, initial_oo])
  at_o, at_h, at_c = LSTM(activation_size, activation = 'tanh', return_state=True,return_sequences=False, dropout = 0.5)(decoder_o, initial_state=[initial_h, initial_c])
  z = Dense(VOCAB_LEN, activation = 'softmax')(at_o)
  print(" Z dimension ", z)
  output.append(z)
  
  print("output Dimensions ", output)
  
  print("at_o: ", at_o)
  print("at_h ", at_h)
  initial_h = at_h
  initial_c = at_c
  initial_o = at_o

model = Model([en_inputs, initial_o_input], output)
print(model.summary())

sgd = optimizers.Adam(lr=0.001)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])



5
(?, ?, 192)
(?, 96)
(?, 96)
(?, 96)
(?, 96)
 o shape of encoder  (?, ?, 192)
 h shape of encoder  (?, 96)
s-1 shape of decoder  Tensor("input_106:0", shape=(?, 96), dtype=float32)
O shape before going into NN  Tensor("concatenate_177/concat:0", shape=(?, 29, 288), dtype=float32)
nn after 20 dense Tensor("dense_20_param_0_36/Relu:0", shape=(?, 29, 20), dtype=float32)
After appplying nuerual network  Tensor("dense_1_param_0_37/truediv:0", shape=(?, 29, 1), dtype=float32)
 o shape before dot  (?, ?, 192)
 context shape after dot  (?, 1, 192)
 Z dimension  Tensor("dense_231/Softmax:0", shape=(?, 40), dtype=float32)
output Dimensions  [<tf.Tensor 'dense_231/Softmax:0' shape=(?, 40) dtype=float32>]
at_o:  Tensor("lstm_182/TensorArrayReadV3:0", shape=(?, 96), dtype=float32)
at_h  Tensor("lstm_182/while/Exit_2:0", shape=(?, 96), dtype=float32)
O shape before going into NN  Tensor("concatenate_179/concat:0", shape=(?, 29, 288), dtype=float32)
nn after 20 dense Tensor("dense_20_param_1_14/Relu

In [188]:
temp_x = np.zeros(shape = ([m, activation_size]))


#print(temp_x[0][0][:4])
print(y[:,0,:].shape)

model.fit([x,temp_x], [y[:,0,:],y[:,1,:],y[:,2,:],y[:,3,:]], epochs= 50, verbose = 1, validation_split = 0.2)  

(797, 40)
Train on 637 samples, validate on 160 samples
Epoch 1/100
637/637 [==============================] - 24s 38ms/step - loss: 8.6405 - dense_231_loss: 1.5655 - dense_232_loss: 1.4519 - dense_233_loss: 2.8518 - dense_234_loss: 2.7713 - dense_231_acc: 0.8069 - dense_232_acc: 0.8085 - dense_233_acc: 0.1287 - dense_234_acc: 0.1036 - val_loss: 5.7815 - val_dense_231_loss: 0.4901 - val_dense_232_loss: 0.4924 - val_dense_233_loss: 2.3527 - val_dense_234_loss: 2.4463 - val_dense_231_acc: 0.8313 - val_dense_232_acc: 0.8313 - val_dense_233_acc: 0.1750 - val_dense_234_acc: 0.0500
Epoch 2/100
637/637 [==============================] - 3s 5ms/step - loss: 5.6025 - dense_231_loss: 0.4478 - dense_232_loss: 0.4491 - dense_233_loss: 2.3606 - dense_234_loss: 2.3450 - dense_231_acc: 0.8524 - dense_232_acc: 0.8524 - dense_233_acc: 0.1429 - dense_234_acc: 0.1099 - val_loss: 5.5401 - val_dense_231_loss: 0.4642 - val_dense_232_loss: 0.4652 - val_dense_233_loss: 2.2863 - val_dense_234_loss: 2.3244 - va

Prediction 

In [63]:
pred_l = ['23-Dec-1955', '24 Dec 1960', 'December 15 2003', 'January of 15 90', '1995 1 Apr', 
          '15/01/2000', '19 May 1999', '01-02-90', '2 October 1993', '25 05 83']

pred_l = [x.lower() for x in pred_l]

#print(pred_l)
pred_m = len(pred_l)
print("TOtal size of prediction list: ", pred_m)

pred_x = np.zeros(shape = (pred_m, MAX_INPUT_SEQ_LEN, VOCAB_LEN))

for i in range(len(pred_l)):
    for j,k in enumerate(pred_l[i]):
      try:
        pred_x[i][j][char_to_ix[k]] = 1
      except:
        print("i {} j {} k {}".format( i, j, k))
        
 

TOtal size of prediction list:  10


In [187]:
o_input = np.zeros(shape =(pred_m,activation_size))
prediction = model.predict([pred_x,o_input])
print(len(prediction))
print(prediction[0].shape)


for i in range(pred_m):
  dates = []
  for j in range(OUTPUT_SIZE):
    pos = np.argmax(prediction[j][i])
    
    
    #char = [ix_to_char[k] for k in pos]
    dates.append(ix_to_char[pos])
  #dates.join(' ')
  print(dates)


4
(10, 40)
[':', '1', '3', '/']
['\t', '-', 'b', '\n']
['r', '1', 'r', ':']
['\t', 'j', 'n', '\n']
['\t', 's', '-', '3']
['-', '-', 'r', 'm']
['m', 's', 'b', '3']
['-', '1', 'r', 'm']
['p', 'j', 'b', '\n']
['p', 's', '\t', '\n']


Let's define the prediction modelling

Prediction will have two Modes - Encoder model and decoder model.

Encoder model will predict the final tensor representing the input text.
Predicted tensor from Encoder will be used as input for decoder model. 

First cell of decoder will have final states from encoder model and input as zero or <SOS> token. The prediction from first cell is our predicted character. The h,s from this cell will be used input to subsequent cell till desired length or <EOS> token is reached.